# Imperfect information games and RL
During this project I\`m going to investigtes difference between  perfect information and imperfect information games.

The main problem of imperfect information games is that you can not forget about previous states like in chess. So in chess you are given postition on board and all privios moves does not metter, because give only this information you could do good move. On that other hand there is a poker which is imperfect information game. Giving that you are on river(final card). You have to know all previous moves of other users because it defi
nes the probability of state that you are in. So to find best tactic in any game position you should solve all tree and is $10^{161}$ states to solve.
    
Optimal strategy for zero-sum game with two players is Nash equilibrium. Nash equilibrium is state in which each player playing his optimal strategy and deviation from this strategies will not make it better. So if you play nesh nash equilibrium strategy you will not lose(but you not abusing mistakes of other players). For finite game with two players ther is guaranteed to exist nash equilibrium.

Current state-of-art poker bot "Libratus" is simplifying game, solving simple version and then extrapoleting this result on bigger game. This is done because to solve game you should solve all tree and this is not feasible. So right now they are doing somethis similar to DeepBlue. This algorims is running on supercomputer

For more detail see
<a href="http://www.youtube.com/watch?feature=player_embedded&v=2dX0lwaQRX0
" target="_blank"><img src="http://img.youtube.com/vi/2dX0lwaQRX0/0.jpg" 
alt="IMAGE ALT TEXT HERE" width="480" height="360" border="10" /></a>

Is scope of this project I will work with [Kuhn poker](https://en.wikipedia.org/wiki/Kuhn_poker). This is simpified version of poker. In Kuhn poker, the deck includes only three playing cards, for example a King, Queen, and Jack. One card is dealt to each player, which may place bets similarly to a standard poker. If both players bet or both players pass, the player with the higher card wins, otherwise, the betting player wins. In scope if this work i will try to find Nash equilibrium for this game
    

In [599]:
#!/bin/python
from roomai.kuhn import *;
import roomai
import numpy as np

import random
from datetime import datetime

from __future__ import division

debug = False

action_bet = 'bet'
action_check = 'check'
state_space = [(0, None, 0), (0, None, 1), (0, None, 2),  
               (1, action_bet, 0), (1, action_bet, 1), (1, action_bet, 2), 
               (1, action_check, 0), (1, action_check, 1), (1, action_check, 2),
               (2, action_bet, 0), (2, action_bet, 1), (2, action_bet, 2), 
               (2, action_check, 0), (2, action_check, 1), (2, action_check, 2)]


n_states = len(state_space)
action_space = [action_bet, action_check]
n_actions = len(action_space)


In [600]:
class DummyActionPlayer(roomai.common.AbstractPlayer):
    """
    Implementtion of Kuhn poker player that play only one card
    """
    def receive_info(self, info):
        self.state = get_state(info)
        pass
    
    def __init__(self, action):
        self.action = action
        
    def take_action(self):
        if debug:
            print "dummy user state ", self.state
        return roomai.kuhn.KuhnPokerAction.lookup(self.action)
        
    def reset(self):
        pass

class AgresivePlayer(roomai.common.AbstractPlayer):
    """
    Kuhn poker player that always bet when he have good card
    """
    def receive_info(self, info):
        self.number = info.person_state.number 
        self.state = get_state(info)
            
    def take_action(self):
        if debug:
            print "Agresive user state ", self.state
        if self.number >= 1:
            return roomai.kuhn.KuhnPokerAction.lookup("bet")
        else:
            return roomai.kuhn.KuhnPokerAction.lookup("check")
        
    def reset(self):
        pass
    

In [601]:
def figth_players(player1, player2, iterations=30000):
    players = [player1] + [player2] + [roomai.common.RandomPlayerChance()]
    env = KuhnPokerEnv()
    total_scores = [0,0]
    for i in range(iterations):
        scores = KuhnPokerEnv.compete(env, players)
        total_scores[0] += scores[0]
        total_scores[1] += scores[1]
    return [s/iterations for s in total_scores]

def get_state(info):
    prev_action = None
    for action in info.public_state.action_history[::-1]:
        if action[0] != 2:
            prev_action = action[1].key
            break
            
    return (info.public_state.epoch, prev_action, info.person_state.number)

check_actor = DummyActionPlayer("check")
bet_actor =  DummyActionPlayer("bet")
agresive_actor = AgresivePlayer()
random_actor = roomai.common.RandomPlayer()

In [602]:
def rand_theta():
	return np.array([random.uniform(0, 1) for j in range(n_states)])

def action_from_prob(prob):
    random.seed(datetime.now())
    return np.random.choice([action_bet, action_check], p=[prob, 1-prob])

class GeneticPlayer(roomai.common.AbstractPlayer):
    """
    Player that uses genetic algorim ro find optimal strategy
    """
    def receive_info(self, info):
        self.number = info.person_state.number 
        self.state = get_state(info)
        scores = info.public_state.scores
        if scores:
            reward = scores[info.person_state.id]
            self.policy_rewards[self.current_policy] += reward
            self.current_policy = (self.current_policy + 1) % self.children_count
            self.iteration += 1
            if self.iteration >= 400:
                self.iteration = 0
                self.update_policies()
    
    def update_policies(self):
        #print self.policy_rewards
        indexs = np.argsort(self.policy_rewards)[::-1]
        #print indexs
        best4 = [self.children_policies[indexs[i]] for i in range(4)]
        muration4 = []
        for _ in range(4):
            perents = random.sample(best4, 2)
            muration4.append([ (perents[0][i]+perents[1][i])/2 for i in range(n_states) ])
        
        self.children_policies = best4 + muration4 + [rand_theta() for _ in range(2)]
        self.policy_rewards = [0 for _ in range(self.children_count)]
        
    def __init__(self):
        self.iteration = 0
        self.children_count = 10
        self.children_policies = [rand_theta() for _ in range(self.children_count)]
        self.current_policy = 0
        self.policy_rewards = [0 for _ in range(self.children_count)]
            
    def take_action(self):
        state_index = state_space.index(self.state)
        action_prob = self.children_policies[self.current_policy][state_index]
        policy_action = action_from_prob(action_prob)
        if debug:
            print "action!! state", self.state
            #print action_prob
            #print policy_action
        return roomai.kuhn.KuhnPokerAction.lookup(policy_action)
        
    def reset(self):
        pass
    




In [603]:
class PolicyPlayer(roomai.common.AbstractPlayer):
    def receive_info(self, info):
        self.state = get_state(info)
    
    def __init__(self, policy):
        self.policy = policy
        
    def take_action(self):
        state_index = state_space.index(self.state)
        action_prob = self.policy[state_index]
        policy_action = action_from_prob(action_prob)
        if debug:
            print "PolicyPlayer state", self.state
            print action_prob
            print policy_action
        return roomai.kuhn.KuhnPokerAction.lookup(policy_action)
        
    def reset(self):
        pass
    
geneticPlayer = GeneticPlayer()
debug = False
for _ in range(10000):
    figth_players(geneticPlayer, check_actor, 1)
    figth_players(geneticPlayer, bet_actor, 1)
    figth_players(geneticPlayer, agresive_actor, 1)
    figth_players(geneticPlayer, random_actor, 1)



## Optimal strategy
Nash equilibrium mixed-strategy for Kuhn_poker taken from [wiki](https://en.wikipedia.org/wiki/Kuhn_poker).

In [604]:
#https://en.wikipedia.org/wiki/Kuhn_poker
nash_equilibrium = [0.33, 0,    1,  
                    0,    0,    1, 
                    0.33, 0.33, 1,
                    0.33, 0.66, 1, 
                    0.33, 0,    1]



In [605]:
best_policy = geneticPlayer.children_policies[0]
print 'genetic vs nash', figth_players(PolicyPlayer(best_policy), PolicyPlayer(nash_equilibrium), 10000)

genetic vs nash [-0.2212, 0.2212]


From wiki we know that the first player should expect to lose at a rate of ~ −1/18 per hand 
So we found close to near strategy